In [1]:
%pip install tonic

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 210 kB 4.7 MB/s eta 0:00:01
     |████████████████████████████████| 8.9 MB 7.6 MB/s eta 0:00:01
     |████████████████████████████████| 292 kB 41.4 MB/s eta 0:00:01     |████████████████████▏           | 184 kB 41.4 MB/s eta 0:00:01
  Created wheel for tonic: filename=tonic-0.2.2-py3-none-any.whl size=31776 sha256=c84ed47a7708b74c3d0934176049650f2201f615b354cb34399e6365c3ca280c
  Stored in directory: /home/laurent/.cache/pip/wheels/69/93/13/d27b27a5d52154ab14788123243f19eaf83ffeece4a4c6ecce
  Created wheel for loris: filename=loris-0.5.0-cp38-cp38-linux_x86_64.whl size=650482 sha256=47e097f1aa603ec33a04e1700808a6027ee3b2ac90b439d9b63b2bfa7f95efcc
  Stored in directory: /home/laurent/.cache/pip/wheels/4f/4b/41/2832ce2dc22c077c5c578e3768e97d91909753bea14fcf16ec
Successfully built tonic loris
Note: you may need to restart the kernel to use updated packages.


In [14]:
import tonic
import tonic.transforms as transforms

testset = tonic.datasets.NMNIST(save_to='./data', train=False)

In [21]:
testset.sensor_size, testset.ordering

((34, 34), 'xytp')

In [22]:
testloader = tonic.datasets.DataLoader(testset,
                                       batch_size=10,
                                       collate_fn=tonic.utils.pad_events,
                                       shuffle=True)

events, target = next(iter(testloader))

In [23]:
target

[9, 8, 4, 8, 8, 7, 1, 3, 2, 0]

In [24]:
events.shape

(10, 5905, 4)

In [28]:
from tonic.utils import plot_events
plot_events(events, sensor_size=testset.sensor_size, ordering=testset.ordering)

AttributeError: module 'tonic.transforms' has no attribute 'MaskIsolated'

In [ ]:
events[0, 0, :]

In [ ]:
events[0, :, 0]

## event representations : 

https://tonic.readthedocs.io/en/latest/transformations.html#event-representations

In [11]:


transform = transforms.Compose([transforms.Denoise(time_filter=10000),
                                transforms.ToSparseTensor()]) 

testset = tonic.datasets.NMNIST(save_to='./data',
                                train=False,
                                transform=transform)

testloader = tonic.datasets.DataLoader(testset, shuffle=True)
events, target = next(iter(testloader))

Using downloaded and verified file: ./data/nmnist_test.zip
Extracting ./data/nmnist_test.zip to ./data


In [11]:


transform = transforms.ToSparseTensor()

testset = tonic.datasets.NMNIST(save_to='./data',
                                train=False,
                                transform=transform)

testloader = tonic.datasets.DataLoader(testset, shuffle=True)
events, target = next(iter(testloader))

Using downloaded and verified file: ./data/nmnist_test.zip
Extracting ./data/nmnist_test.zip to ./data


In [12]:
events.shape

torch.Size([1, 299306, 26, 34])